<a href="https://colab.research.google.com/github/hemanthkumar17/discFuncClassification/blob/main/Discriminant_function_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classification using Linear Discriminant function

A Linear Discriminant function is a f

$
g(x) = -\frac{1}{2}(x-\mu_i)^t\sum_i^{-1}(x-\mu_i)\ 
-\ \frac{d}{2}ln\ 2\pi\ 
-\ \frac{1}{2}ln\ |\sum_i|\ 
+\ ln\ P(\omega_i)
$

In [1]:
import numpy as np

In [96]:
# numpy functions are used to calculate the discriminant functions
def df(x, n_feat, w_mean, w_cov, w_prior):
    func = {}
    for i, (i_mean, i_cov, i_prior) in enumerate(zip(w_mean, w_cov, w_prior)):
        if w_prior != 0:
            func[f"w{i}"] = -1/2 * np.matmul(np.matmul(np.transpose(x - i_mean), np.linalg.inv(i_cov)), (x - i_mean))
            func[f"w{i}"] -= n_feat/2 * np.log(2 * np.pi)
            func[f"w{i}"] -= 1/2 * np.log(np.linalg.det(i_cov)) 
            func[f"w{i}"] += np.log(i_prior) 
        else:
            # If the prior probabilities are 0, the function returns the least possible value so that the classification
            # will lean towards the other classes
            func[f"w{i}"] = -np.inf
    return func

The Data is defined and the mean, covariances are calculated.<br>
The prior probabiblities are defined based on the assumption.

In [71]:
# Given datapoints on which the discriminant function learns the classification
DATA = DATA = {
    # W1
    "w1": np.array([
        [-5.01, -8.12, -3.68],
        [-5.43, -3.48, -3.54],
        [1.08, -5.52, 1.66],
        [0.86, -3.78, -4.11],
        [-2.67, 0.63, 7.39],
        [4.94, 3.29, 2.08],
        [-2.51, 2.09, -2.59],
        [-2.25, -2.13, -6.94],
        [5.56, 2.86, -2.26],
        [1.03, -3.33, 4.33]
    ]),

    # W2
    "w2": np.array([
        [-0.91, -0.18, -0.05],
        [1.30, -2.06, -3.53],
        [-7.75, -4.54, -0.95],
        [-5.47, 0.50, 3.92],
        [6.14, 5.72, -4.85],
        [3.60, 1.26, 4.36],
        [5.37, -4.63, -3.65],
        [7.18, 1.46, -6.66],
        [-7.39, 1.17, 6.30],
        [-7.50, -6.32, -0.31]
    ]),

    # W3
    "w3": np.array([
        [5.35, 2.26, 8.13],
        [5.12, 3.22, -2.66],
        [-1.34, -5.31, -9.87],
        [4.48, 3.42, 5.19],
        [7.11, 2.39, 9.21],
        [7.17, 4.33, -0.98],
        [5.75, 3.97, 6.65],
        [0.77, 0.27, 2.41],
        [0.90, -0.43, -8.71],
        [3.52, -0.36, 6.43]
    ]) }

# The mean and covariances for each class based on the data are defined
w_mean = np.array([np.mean(DATA[i], axis=0) for i in DATA])
w_cov = np.array([np.cov(np.transpose(DATA[i])) for i in DATA])

# Output code to view the values calculated
for i, mean in enumerate(w_mean):
    print(f"Mean for class w{i}:")
    print(mean)
for k, cov in enumerate(w_cov):
    print("-"*20 + f"\nCovariance for class w{k}\n" + "-"*20)
    for i, _ in enumerate(cov):
        for j, ij_cov in enumerate(_):
            if i == j:
                s = f"Variance of x{i}"
            else:
                s = f"Covariance of x{i} and x{j}:"
            print(s)
            print(ij_cov)

Mean for class w0:
[-0.44  -1.749 -0.766]
Mean for class w1:
[-0.543 -0.762 -0.542]
Mean for class w2:
[3.883 1.376 1.58 ]
--------------------
Covariance for class w0
--------------------
Variance of x0
14.38051111111111
Covariance of x0 and x1:
7.695377777777775
Covariance of x0 and x2:
4.122322222222222
Covariance of x1 and x0:
7.695377777777775
Variance of x1
14.623121111111107
Covariance of x1 and x2:
3.90684
Covariance of x2 and x0:
4.122322222222222
Covariance of x2 and x1:
3.90684
Variance of x2
19.724537777777776
--------------------
Covariance for class w1
--------------------
Variance of x0
36.82933444444444
Covariance of x0 and x1:
9.980926666666667
Covariance of x0 and x2:
-16.36675111111111
Covariance of x1 and x0:
9.980926666666667
Variance of x1
13.16855111111111
Covariance of x1 and x2:
0.40905111111111125
Covariance of x2 and x0:
-16.36675111111111
Covariance of x2 and x1:
0.40905111111111125
Variance of x2
18.421217777777773
--------------------
Covariance for class 

Q2. Calculate the function values to find which class the data should be classified into using the discriminant function and the features we have to take.

In [150]:
# Prior is defined as provided in the assumption
w_prior = [1/2, 1/2, 0]
# The d refers to the dimension of the feature space
def classify(d, dataPoints = DATA):
    print("="*20)
    # We are taking the features from 1 to d to calculate the discriminant function value
    features = [f"x{x+1}" for x in list(range(d))]
    print(f"For Features:{features}")
    overall = 0

    # If the data is not given, it defaults to DATA thus implying that we are verifying the classification of the function
    # to the ground truth
    calcError = True
    if type(dataPoints) != dict:
        calcError = False
        dataPoints = {"Not classified": dataPoints}
    # Loop through each classes available (keys)
    for i in dataPoints:
        print("-"*20)
        if calcError:
            correct = 10
            index = int(i[1]) - 1
            print(f"\nClass {i}\n" + "-"*20)
        # We find the discriminant for each datapoint in the class w{i}
        for data in dataPoints[i]:
            dfVal = df(data[0:d], n_feat = d, w_mean=w_mean[:, 0:d], w_cov=w_cov[:, 0:d, 0:d], w_prior=w_prior)
            # We find the maximum value of all the discriminant functions calculated
            val = np.argmax(np.array(list(dfVal.values())))
            print(f"The data {data}\t is classified as w{val+1}")
            if calcError and val != index:
                correct -= 1
        # Printing the error percentage between the function and the ground truth
        if calcError:
            print(f"Percentage of points misclassified = {100 - correct*10} %")
            overall += 100 - correct*10
    if calcError:
        print("+"*20 + f"\nOverall Percentage of values misclassified = {np.round(overall/3, 2)} %\n" + "+"*20)

Q2.a and b We take only the features x1 and find the discriminant values to classify the data points and calculate the error percentage for each class as well as overall error

In [151]:
classify(1)

For Features:['x1']
--------------------

Class w1
--------------------
The data [-5.01 -8.12 -3.68]	 is classified as w1
The data [-5.43 -3.48 -3.54]	 is classified as w2
The data [ 1.08 -5.52  1.66]	 is classified as w1
The data [ 0.86 -3.78 -4.11]	 is classified as w1
The data [-2.67  0.63  7.39]	 is classified as w1
The data [4.94 3.29 2.08]	 is classified as w2
The data [-2.51  2.09 -2.59]	 is classified as w1
The data [-2.25 -2.13 -6.94]	 is classified as w1
The data [ 5.56  2.86 -2.26]	 is classified as w2
The data [ 1.03 -3.33  4.33]	 is classified as w1
Percentage of points misclassified = 30 %
--------------------

Class w2
--------------------
The data [-0.91 -0.18 -0.05]	 is classified as w1
The data [ 1.3  -2.06 -3.53]	 is classified as w1
The data [-7.75 -4.54 -0.95]	 is classified as w2
The data [-5.47  0.5   3.92]	 is classified as w2
The data [ 6.14  5.72 -4.85]	 is classified as w2
The data [3.6  1.26 4.36]	 is classified as w1
The data [ 5.37 -4.63 -3.65]	 is classif

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in log
  


Q2.c We repeat the same as above using 2 features, classifying using x1 and x2

In [152]:
classify(2)

For Features:['x1', 'x2']
--------------------

Class w1
--------------------
The data [-5.01 -8.12 -3.68]	 is classified as w1
The data [-5.43 -3.48 -3.54]	 is classified as w2
The data [ 1.08 -5.52  1.66]	 is classified as w1
The data [ 0.86 -3.78 -4.11]	 is classified as w1
The data [-2.67  0.63  7.39]	 is classified as w2
The data [4.94 3.29 2.08]	 is classified as w2
The data [-2.51  2.09 -2.59]	 is classified as w2
The data [-2.25 -2.13 -6.94]	 is classified as w1
The data [ 5.56  2.86 -2.26]	 is classified as w2
The data [ 1.03 -3.33  4.33]	 is classified as w1
Percentage of points misclassified = 50 %
--------------------

Class w2
--------------------
The data [-0.91 -0.18 -0.05]	 is classified as w1
The data [ 1.3  -2.06 -3.53]	 is classified as w1
The data [-7.75 -4.54 -0.95]	 is classified as w2
The data [-5.47  0.5   3.92]	 is classified as w2
The data [ 6.14  5.72 -4.85]	 is classified as w2
The data [3.6  1.26 4.36]	 is classified as w1
The data [ 5.37 -4.63 -3.65]	 is c

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in log
  


Q2. d We repeat it using features x1, x2 and x3

In [153]:
classify(3)

For Features:['x1', 'x2', 'x3']
--------------------

Class w1
--------------------
The data [-5.01 -8.12 -3.68]	 is classified as w1
The data [-5.43 -3.48 -3.54]	 is classified as w1
The data [ 1.08 -5.52  1.66]	 is classified as w1
The data [ 0.86 -3.78 -4.11]	 is classified as w1
The data [-2.67  0.63  7.39]	 is classified as w2
The data [4.94 3.29 2.08]	 is classified as w1
The data [-2.51  2.09 -2.59]	 is classified as w1
The data [-2.25 -2.13 -6.94]	 is classified as w1
The data [ 5.56  2.86 -2.26]	 is classified as w2
The data [ 1.03 -3.33  4.33]	 is classified as w1
Percentage of points misclassified = 20 %
--------------------

Class w2
--------------------
The data [-0.91 -0.18 -0.05]	 is classified as w2
The data [ 1.3  -2.06 -3.53]	 is classified as w2
The data [-7.75 -4.54 -0.95]	 is classified as w2
The data [-5.47  0.5   3.92]	 is classified as w2
The data [ 6.14  5.72 -4.85]	 is classified as w2
The data [3.6  1.26 4.36]	 is classified as w1
The data [ 5.37 -4.63 -3.65]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in log
  


Q2. d <br>
Error rates:<br>

| Features    | Error rate |
| ----------- | ---------- |
| x1          | 53.33 %    |
| x1, x2      | 63.33 %    |
| x1, x2, x3  | 43.33 %    |


From the results, the misclassification rate is lowest when taking all 3 features and highest when taking x1 and x2.<br>
This happens because the covariance between the features in the case of w3 is much higher when compared to w1 and w2

Q2. f <br>
We calculate the discriminant values for the data points given in the 3 methods discussed above(x1 only, x1 and x2, all 3) 

In [154]:
newData = [
           [1, 2, 1],
           [5, 3, 2],
           [0, 0, 0],
           [1, 0, 0]
]
for i in range(1, 4):
    classify(i, np.array(newData))

For Features:['x1']
--------------------
The data [1 2 1]	 is classified as w1
The data [5 3 2]	 is classified as w2
The data [0 0 0]	 is classified as w1
The data [1 0 0]	 is classified as w1
For Features:['x1', 'x2']
--------------------
The data [1 2 1]	 is classified as w1
The data [5 3 2]	 is classified as w2
The data [0 0 0]	 is classified as w1
The data [1 0 0]	 is classified as w1
For Features:['x1', 'x2', 'x3']
--------------------
The data [1 2 1]	 is classified as w2
The data [5 3 2]	 is classified as w1
The data [0 0 0]	 is classified as w1
The data [1 0 0]	 is classified as w1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in log
  
